In [0]:
def mount_adls(storage_account_name, container_name):
    # Get secrets from Key Vault
    client_id = dbutils.secrets.get(scope='tokyo-olympic-scope', key = 'client-id-tokyo-olympic')
    tenant_id = dbutils.secrets.get(scope='tokyo-olympic-scope', key = 'tenant-id-tokyo-olympic')
    client_secret = dbutils.secrets.get(scope='tokyo-olympic-scope', key = 'client-secret-tokyo-olympic')
    
    # Set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
              "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
              "fs.azure.account.oauth2.client.id": client_id,
              "fs.azure.account.oauth2.client.secret": client_secret,
              "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Unmount the mount point if it already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")
    
    # Mount the storage account container
    dbutils.fs.mount(
      source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
      mount_point = f"/mnt/{storage_account_name}/{container_name}",
      extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls('kkdtokyoolympicdata2021','tokyo-olympic-data') 

/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/tokyoolymic,abfss://tokyo-olympic-data@kkdtokyoolympicdata2021.dfs.core.windows.net,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data,abfss://tokyo-olympic-data@kkdtokyoolympicdata2021.dfs.core.windows.net/,
/volumes,DbfsReserved,
/,DatabricksRoot,


In [0]:
display(dbutils.fs.ls("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data"))

path,name,size,modificationTime
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/,processed/,0,1701283276000
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/,raw/,0,1701283269000


In [0]:
%fs
ls "dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/"

path,name,size,modificationTime
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/athletes.csv,athletes.csv,418492,1701288551000
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/coaches.csv,coaches.csv,16889,1701288564000
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/emtries_gender.csv,emtries_gender.csv,1123,1701288575000
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/medals.csv,medals.csv,2414,1701288586000
dbfs:/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/teams.csv,teams.csv,35270,1701288600000


In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, DateType, BooleanType 

In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/coaches.csv")
entried_gender = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/emtries_gender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/raw/teams.csv")


In [0]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [0]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entried_gender.withColumn(
    'Avg_Female', entried_gender['Female'] / entried_gender['Total']
).withColumn(
    'Avg_Male', entried_gender['Male'] / entried_gender['Total']
)
average_entries_by_gender.show()
     

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').parquet("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/athletes")

In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").parquet("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/coaches")
entried_gender.repartition(1).write.mode("overwrite").option("header","true").parquet("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").parquet("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").parquet("/mnt/kkdtokyoolympicdata2021/tokyo-olympic-data/processed/teams")